In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pyodbc
import pandas as pd

In [2]:
#Insert the rows of the fresh file into table of the sqlserver

In [3]:
cnxn = pyodbc.connect('Driver={SQL Server};Server=KTCL-5CG9362KJQ\SQLEXPRESS;Database=GBSS;Trusted_Connection=yes;')
cursor = cnxn.cursor()
url='https://covid.ourworldindata.org/data/ecdc/full_data.csv'
df = pd.read_csv(url)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59354 entries, 0 to 59353
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   date             59354 non-null  object 
 1   location         59354 non-null  object 
 2   new_cases        59021 non-null  float64
 3   new_deaths       59021 non-null  float64
 4   total_cases      56051 non-null  float64
 5   total_deaths     46414 non-null  float64
 6   weekly_cases     58222 non-null  float64
 7   weekly_deaths    58222 non-null  float64
 8   biweekly_cases   56717 non-null  float64
 9   biweekly_deaths  56717 non-null  float64
dtypes: float64(8), object(2)
memory usage: 4.5+ MB


In [5]:
#to save the fresh csv file

In [6]:
url='https://covid.ourworldindata.org/data/ecdc/full_data.csv'
#to download the csv file from url and convert it to dataframe
df = pd.read_csv(url)
#to replace the contents of the file with fresh data 
df.to_csv('full_data.csv',index=False,header=True)

In [7]:
#Function that downloads,replaces the file and database table with the fresh data
#the below function does all the required stuff

In [8]:
def insert_fresh_data():
    cnxn = pyodbc.connect('Driver={SQL Server};Server=KTCL-5CG9362KJQ\SQLEXPRESS;Database=GBSS;Trusted_Connection=yes;')
    cursor = cnxn.cursor()
    
    cursor.execute('''TRUNCATE TABLE [dbo].[full_data (2)]''') # removes the data of the table except column names               
    cnxn.commit()

    cursor=cnxn.cursor()
    url='https://covid.ourworldindata.org/data/ecdc/full_data.csv'
    df = pd.read_csv(url)
    df=df.fillna(0)
    df.to_excel('latest_covid_data.xlsx',index=False,header=True)
    df.to_csv('full_data2.csv',index=False,header=True)  #replace the fresh data into file
    for rows in df.itertuples(index=False):             # Iterate through each row elementof dataframe
        cursor.execute("INSERT INTO [dbo].[full_data (2)] VALUES(?,?,?,?,?,?,?,?,?,?)",rows)
    cnxn.commit()
insert_fresh_data()

In [9]:
#SQL QUERIES 

#TO PREPARE A CTE OF DATA TO DO MORE CALCULATION
with ccte as 
(
 select [location] as loc,max([date]) as date,max([total_cases]) as tc,max([new_cases]) as nc,max([total_deaths]) as td,max([new_deaths]) as nd
 from [dbo].[full_data (2)]
 where [location]!='World'
 group by [location]
)
select sum(td) as totaldeaths,sum(nd) as new_d,sum(tc) as tot_c,sum(nc) as new_c from ccte

# TO GET MAXIMUM DEATH TILL NOW
select [location] as loc,max([date]) as date,max([total_cases]) as tc,max([new_cases]) as nc,max([total_deaths]) as td,max([new_deaths]) as nd
from [dbo].[full_data (2)]
where [location]!='World'
group by [location]
order by loc

#TO 
select [location] as loc,[date],[total_cases],[new_cases],[total_deaths],[new_deaths]
from [dbo].[full_data (2)]
where [date]=(select max([date]) from [dbo].[full_data (2)])


SyntaxError: invalid syntax (<ipython-input-9-be3747667dcd>, line 4)